In [1]:
from scrape_modules import ScrapeIshikawa
import requests, json
import slackweb
json_open = open('important.json', 'r')
important_list = json.load(json_open)

In [2]:
# データスクレイピング
sps_url = important_list['sps_url']
si = ScrapeIshikawa(sps_url)
driver = si.open_driver(False)
sikyou_url_list = si.scrape_sikyou_url(driver)
sikyou_url_list_new = si.get_sikyou_url_new(sikyou_url_list)
si.scrape_sikyou_all(driver, sikyou_url_list_new)
# si.scrape_sikyou_all(driver, sikyou_url_list)
driver.quit()

In [3]:
# 魚種列，目方列追加
df_species = si.scrape_data['銘　柄'].map(lambda x: si.container2species_size(x)[0])
df_size = si.scrape_data['銘　柄'].map(lambda x: si.container2species_size(x)[1])
si.scrape_data.insert(3, '魚種',df_species)
si.scrape_data.insert(4, '目方',df_size)

In [4]:
# データ結合
si.concat_new_data()

In [5]:
# データ確認
print(len(si.sps_data))
print(len(si.scrape_data))
print(len(si.sps_data_new))
si.sps_data_new

393
0
393


,日付,漁 法,銘 柄,魚種,目方,卸売数量[kg],高値[円/kg],平均[円/kg],安値[円/kg],1箱重量[kg]
0,2020-12-13,底曳網,あかがい(ｴｿﾞﾎﾞﾗﾓﾄﾞｷ),あかがい,-,45,460,320,120,5
1,2020-12-13,いか釣,するめいか(ﾊﾞﾗ),するめいか,ﾊﾞﾗ,235,700,500,300,5
2,2020-12-13,底曳網,あかがれい,あかがれい,-,384,1167,872,333,3
3,2020-12-13,その他,べにずわいがに,べにずわいがに,-,231,567,567,567,-
4,2020-12-13,その他,さめ類,さめ類,-,12,167,125,83,6
...,...,...,...,...,...,...,...,...,...,...
388,2020-12-02,底曳網,加能ガニ(ｽﾞﾜｲｶﾞﾆ),加能ガニ,-,136,8750,4905,375,4
389,2020-12-02,底曳網,加能ガニ10入(ｽﾞﾜｲｶﾞﾆ),加能ガニ,10入,80,15000,11147,6000,4
390,2020-12-02,底曳網,加能ガニ4入(ｽﾞﾜｲｶﾞﾆ),加能ガニ,4入,172,20750,14219,7750,4
391,2020-12-02,底曳網,まだら,まだら,-,50,1900,940,400,5


In [6]:
# データ保存
si.save_sps('市況')

In [7]:
# 魚種に対して全サイズの平均価格を算出。
# 魚種を列，日付を行としたデータフレームを作成。
df_per_day = si.merge_per_day()
df_per_day.head(5)

,日付,あかがい,あかがれい,かわはぎ,がすえび,するめいか,なめたがれい,ばいがい,ほっけ,まだい,...,ひらめ,がんど活〆,きんめだい,くろだい,はたはた,まあじ,あら,さこだい,さめ類,ぶり活〆
0,2020-12-02,552.0,1259.0,570.5,2295.0,990.000000,560.0,1160.0,500.0,1258.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-03,NaN,NaN,NaN,NaN,793.500000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-12-04,NaN,NaN,1500.0,1952.0,948.333333,722.0,NaN,312.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-12-06,597.0,1165.0,NaN,3038.0,681.600000,469.0,779.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-12-07,499.0,1005.0,1062.5,1665.0,630.200000,358.0,576.0,400.0,892.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# データ保存
si.set_with_df("by_species", df_per_day)

In [9]:
# 魚種に対して全サイズの平均価格を算出し，データフレームに格納
df_per_day_and_species = si.merge_all_per_ds()
df_per_day_and_species.head(5)
df_per_day_and_species.fillna(0, inplace=True)

In [10]:
# データ保存
si.set_with_df("by_species_size", df_per_day_and_species)

In [11]:
# slackに通知
content = {
    "icon_url" : "https://drive.google.com/uc?export=view&id=1bbPewWm7dHriVWqHQ2IJe3uzwbN1-RHE",
    'username': "JF石川市況",
    "text": f"JF石川の市況データを更新しました\n{important_list['sps_url']}"
    }

webhook_url = important_list["slack_webhook_url"]
requests.post(webhook_url, data = json.dumps(content))

<Response [200]>